# Assignment 9

Name 1: <br/>
Student id 1: <br/>
Email 1: <br/>


Name 2: <br/>
Student id 2: <br/>
Email 2: <br/> 

**Instructions:** Read each question carefully. <br/>
Make sure you appropriately comment your code wherever required. Your final submission should contain the completed Notebook and the Python files. There is no need to submit the data files. <br/>
Upload the zipped folder in Teams. Make sure to click on "Turn-in" after your upload your submission, otherwise the assignment will not be considered as submitted. Only one from the group should make the submisssion.

---

# Exercise 1: Text Classification (10 points)

Based on your implementation of the `Corpus` and `Document` classes from the previous assignment, you will now build a simple Naive Bayes classifier to classify each document in the test section of the Reuters News corpus. 

We will use the TF-IDF metric as the feature for our classifier. TF-IDF of a term $t$ in a document $d$ is defined as:

\begin{equation}
  \text{TF-IDF}(t,d) = \frac{
    \text{TF}(t,d)
  }{
    \text{IDF}(t)
  }
\end{equation}

with $\text{TF}(t,d)$ being the defined as

\begin{equation}
  \text{TF}(t,d) = \frac{
    f_{t,d}
  }{
    \sum_{t'} f_{t',d}
  }
\end{equation}

where $f_{t,d}$ is the absolute frequency of term $t$ in document $d$.

and $\text{IDF}(t)$ being defined as 

\begin{equation}
  \text{IDF}(t) = \frac{
    N
  }{
    |\{d \in D: C_d(t) > 0\}|
  }
\end{equation}

where $D$ stands for the documents in the corpus, $N=|D|$ and $C_d(t)$ is the number of times term $t$ occurs in document $d$.

In a TF-IDF matrix, documents are represented by the rows of the matrix and TF-IDF features by its columns. This means that each row vector consists of the TF-IDF value for a term taken from a fixed, shared vocabulary given the document, i. e. $\text{TF-IDF}(t,d)$, for $t \in V$ ([this](https://www.researchgate.net/profile/Maryam-Hourali/publication/306358542/figure/tbl1/AS:648973966651395@1531738859631/Some-Part-of-TF-IDF-Term-Document-Matrix.png) is a small example). 

## 1.1 Vocabulary as feature space (2 points)

Construct a shared vocabulary $V$ for the Reuters corpus, using both the train set and the test set. You are expected to reduce the size of the vocabulary by 
  * Preprocessing (removing punctuation, lowercasing, tokenizing). (0.25 points)
  * Lemmatizing the tokenized text. (0.5 points)
  * Setting a $\text{min_df}$ and $\text{max_df}$ and removing all terms from the vocabulary that occur in less then $\text{min_df}$ and more than $\text{max_df}$ documents. You should support your choice with a source from the internet or your own reasoning. (0.5 point)
  * Why is it necessary to reduce the size of the vocabulary and to set a lower and upper bound to document frequency? Explain in 2-3 sentences. (0.25 points)

You are allowed to use any Python package useful to the task. We suggest using NLTK's [RegexpTokenizer](https://www.nltk.org/api/nltk.tokenize.html#nltk.tokenize.regexp.RegexpTokenizer) for tokenization and [WordNetLemmatizer](https://www.nltk.org/api/nltk.stem.html#nltk.stem.wordnet.WordNetLemmatizer) for lemmatization. The implementation should be in the `reduce_vocabulary` method of the `Corpus` class. Check that your implementation is correct by executing the code cell below and comparing vocabulary sizes before and after the reduction. 
  
As always, you are free to define new methods as you need them.  

**Answers**
- 
- It is important to reduce the vocabulary size in order to preserve only words that provide valuable information for the task. Words that appear in many or all documents are not helpful for distinguishing between different document categories. On the other hand, words that appear only in a few documents (e.g. 1 or 2) are probably not representative features of that document category. Lastly, due to computational limitations, it might be required to reduce the vocabulary size.

In [12]:
# Data loading
from nltk.corpus import reuters, stopwords
import nltk
nltk.download('wordnet')
stop_words = stopwords.words('english')

[nltk_data] Downloading package wordnet to /home/pavle/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


In [37]:
from importlib import reload
import exercise_1
exercise_1 = reload(exercise_1)
from tqdm import tqdm

print("Loading Reuters corpus...")
corpus = exercise_1.Corpus(
    documents=[
    exercise_1.Document(fileid, reuters.raw(fileid), reuters.categories(fileid), stop_words=stop_words) 
    for fileid in tqdm(reuters.fileids())],
    categories=reuters.categories()
)
print("\nVocab size before reduction:", len(corpus.terms()))

# TODO: set min_df, max_df
min_df = 3
max_df = 7

reduced_vocab = corpus.reduce_vocab(min_df=min_df, max_df=max_df)

print("\nVocab size after reduction:", len(reduced_vocab))

100%|██████████| 10788/10788 [00:14<00:00, 744.68it/s]

Vocab size before reduction: 28371

Vocab size after reduction: 6236


## 1.2 TF-IDF matrix (2 points)

1. Implement the method `_idfs` of the `Corpus` class. It should take the reduced vocabulary as input and return a dictionary containing the IDFs of each word in the reduced vocabulary. Print the IDFs of the first 10 terms (sorted lexicographically) from the reduced vocabulary. Store the IDFs in a class variable `idfs`. Why is it a good idea to calculate IDFs first? (1 points)

2. Implement the method `_tfs_idfs` of the corpus class. It should return a vector or a a list containing the TF-IDFs of all terms in the reduced vocabulary for a single document. It should use the `_idfs` method once internally. (1 point).

In [38]:
# TODO: load and print IDFs!
idfs = corpus._idfs(reduced_vocab)
print("Estimating idfs")
for term in reduced_vocab[:10]:
  print(term, idfs[term])

Estimating idfs
0 8.123493975903614
00 51.86538461538461
000 3.546351084812623
007 980.7272727272727
009 1078.8
010 634.5882352941177
02 105.76470588235294
04 96.32142857142857
040 469.04347826086956
05 75.97183098591549


## 1.3 Train/test split (1.5 points)

1. Implement the method `_category2index`. It should take a string (the name of the category) as input and return its index in the `Corpus`-internal list of categories. (0.25 points)
2. Implement the method `compile_dataset` of the `Corpus` class. It should take the reduced vocabulary as input and return two tuples: (train TF-IDF matrix, train labels) and (test TF-IDF matrix, test labels). The train matrix/labels should be derived from the train section of the Reuters dataset (file-ids starting with `training/`) and the test matrix/labels from the test section (file-ids starting with `test/`).

  Make use of the methods `_tf_idfs` and `_category2index` (1 point)

3. Use the method `compile_dataset` to load the train and test data into variables. Please name the variables such that we can distinguish the train data from the test data. Show the size of the train and test set. (0.25 points)

In [39]:
# TODO: load train and test data
(X_train, Y_train), (X_test, Y_test) = corpus.compile_dataset(reduced_vocab)

# Show size of train and test set
print(f"Train data shape: {X_train.shape}, train labels shape: {Y_train.shape}")
print(f"Test data shape: {X_test.shape}, test labels: {Y_test.shape}")

Train data shape: (7769, 6236), train labels shape: (7769,)
Test data shape: (3019, 6236), test labels: (3019,)


## 1.4: Naive Bayes Classifier (5 points)

A Naive Bayes classifier assigns a datapoint $x = x_1,...x_n$ to a class $C_k$ ($1 \leq k \leq K$, with $K$ being the number of classes) with probability $P(C_k|x)$ given by:

\begin{equation}
  p(C_k|x) = \frac{
    p(C_k)p(x|C_k)
  }{
    p(x)
  }
\end{equation}

1.  Describe the idea behind Naive Bayes in 3-4 sentences. Do so by explaining the terms 'naive' and 'Bayes(ian)' (1 point)
2. For each part of the above formula, assign it to one of the following categories, and give a short explanation. (1 point)
  * Prior
  * Posterior
  * Likelihood
  * Evidence

3. In our dataset from 1.3, what corresponds to $C_k$? What to $x$? (0.5 points)

4. What is a good baseline for estimating the accuracy of our classifier? How would you evaluate it? Explain in 1-2 sentences **and** support your answer with code. This will also help you check the accuracy you get on the actual data. (1 point)

5. Train a Naive Bayes classifier on the train section of our dataset and report precision, accuracy and F-score on the test section. You may use the class [GaussianNB](https://scikit-learn.org/stable/modules/generated/sklearn.naive_bayes.GaussianNB.html) and the method [precision_recall_fscore](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.precision_recall_fscore_support.html) from the [scikit-learn](https://scikit-learn.org/stable/install.html) Python package. You can write the code in the code cell below. (2 points)
  
6. Do you observe a difference in the F-scores of different classes? Why? What could you do to account for your finding? (0.5 points)

**Answers** <br>
1. Naive Bayes is collection of algorithms based on Bayes Theorem which assume that all features are independent of each other and they contribute equally to classification. Bayesian approach is a probabilistic approach for updating a prior belief for values given that some event happend. Naive part comes from independence assumption which allows for transforming the formula $P(C_k|x) = \frac{P(C_k)P(x|C_k)}{P(x)}$ to $P(C_k|x) = \frac{P(C_k)P(x_1|C_k)P(x_2|C_k)...P(x_n|C_k)}{P(x_1)P(x_2)...P(x_n)}$. Finally, since $P(x_1)P(x_2)...P(x_n)$ is constant over training and different classes, it can be omitted, and leaving the final simplified formula $P(C_k|x) = P(C_k) \prod_{i=1}^n{P(x_i|C_k)}$. <br>
2. Parts of the formula: <br>
  * Prior - $P(C_k)$ -> It is initial belief for the value of interest.
  * Posterior - $P(C_k|x)$ -> It is updated belief for the value of interest given that some event happend.
  * Likelihood - $P(x|C_k)$ -> It is probability that some event will happen given the value of interest.
  * Evidence - $P(x)$ -> It is probability that some event that cause update of the value of interest happend.
3. $C_k$ is $k^{th}$ category of all posible for documents classification, while $x$ is feature vectors, i.e. reduced vocabulary. <br>
4. <br>
6. <br>

In [ ]:
# TODO: Find accuracy of baseline classifier

# TODO: train classifier, report precision, recall, fscore

## Bonus: Support Vector Machines (1.5 points)

Consider the task of Named Entity Recognition. In a simplified scenario, you want to decide for each word if it belongs to one of the following classes: {not-named-entity, person, city, country, currency}. An expert in the field tells you that you should start with the following set of features:
- is the whole word in capitals
- is the first letter capitalized
- does it begin a sentence
- number of characters 
- is a stopword
- number of Wikipedia articles that contain this word in their title

1. Come up with at least 3 more features for this problem. (0.2 points)
2. How can we numerically represent each datapoint? What is the mathematical object called and what is the set in which it lives? (0.2 points)
3. What is a hyperplane and how can it be used in this context? (0.2 points)
4. Imagine that you've been given two features: $f_1, f_2$ and the following dataset. The task is currently only to distinguish between two classes. Draw the points and 3 hyperplanes:
  - one that mispredicts at least one datapoint
  - one that predicts everything correctly
  - one that predicts everything correctly but is in some sense worse than the previous one

|Data point|$f_1$|$f_2$|class|
|---|---|---|---|
|$d_1$|2|2|Y|
|$d_2$|10|9|Y|
|$d_3$|2|5|Y|
|$d_4$|3|5|Y|
|$d_5$|2|-2|N|
|$d_6$|10|0|N|
|$d_7$|10|-4|N|
|$d_8$|3|3|N|

  In all cases provide the formula for the hyperplane and explain how to use it to make a decision regarding which class it belongs to. (0.65 points)

5. In the previous question, you created hyperplanes that helped you in determining which of the two classes the datapoint belongs to. How would you extend this to solve the original problem, i.e. predicting which of the 5 classes the datapoint belongs to? (0.25 points)